In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
import zipfile
import os
import glob
import random
import sys

import skimage.io  #Used for imshow function
import skimage.transform  #Used for resize function
from skimage.morphology import label  #Used for Run-Length-Encoding RLE to create final submission
import matplotlib.pyplot as plt

from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
# Загрузка данных
data_dir = 'data_256/'
images_dir = 'images/'
masks_dir = 'masks/'
WIDTH = 256
HEIGHT = 256
CLASSES = 1

In [16]:
import os
from PIL import Image
import numpy as np


def load_image(image_path):
    image = Image.open(data_dir + images_dir + image_path)  #.convert('L')  # Преобразуем в оттенки серого
    return np.array(image)


import numpy as np
from PIL import Image

def load_mask(mask_path):
    mask = Image.open(data_dir + masks_dir + mask_path).convert('L')  # Преобразуем в оттенки серого
    mask_array = np.array(mask)
    mask_float = mask_array.astype(float) / 255.0  # Преобразуем в тип float и нормализуем значения до диапазона [0, 1]
    return mask_float


In [17]:
# # Подготовка данных
image_paths = sorted(os.listdir(data_dir + images_dir))

In [18]:
print(image_paths)

['1.png', '10.png', '100.png', '101.png', '102.png', '103.png', '104.png', '105.png', '106.png', '107.png', '108.png', '109.png', '11.png', '110.png', '111.png', '112.png', '113.png', '114.png', '115.png', '116.png', '117.png', '118.png', '119.png', '12.png', '120.png', '121.png', '122.png', '123.png', '124.png', '125.png', '126.png', '127.png', '128.png', '129.png', '13.png', '130.png', '131.png', '132.png', '133.png', '134.png', '135.png', '136.png', '137.png', '138.png', '139.png', '14.png', '140.png', '141.png', '142.png', '143.png', '144.png', '145.png', '146.png', '147.png', '148.png', '149.png', '15.png', '150.png', '151.png', '152.png', '153.png', '154.png', '155.png', '156.png', '157.png', '158.png', '159.png', '16.png', '160.png', '161.png', '162.png', '163.png', '164.png', '165.png', '166.png', '167.png', '168.png', '169.png', '17.png', '170.png', '171.png', '172.png', '173.png', '174.png', '175.png', '176.png', '177.png', '178.png', '179.png', '18.png', '180.png', '181.png'

In [19]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую, валидационную и тестовую выборки
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)
train_paths, valid_paths = train_test_split(train_paths, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Проверка размеров выборок
print("Train samples:", len(train_paths))
print("Valid samples:", len(valid_paths))
print("Test samples:", len(test_paths))

Train samples: 358
Valid samples: 120
Test samples: 120


In [20]:
train_images = [load_image(image_path) for image_path in train_paths]
train_masks = [load_mask(mask_path) for mask_path in train_paths]
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [21]:
valid_images = [load_image(image_path) for image_path in valid_paths]
valid_masks = [load_mask(mask_path) for mask_path in valid_paths]
valid_images = np.array(valid_images)
valid_masks = np.array(valid_masks)

In [22]:
test_images = [load_image(image_path) for image_path in test_paths]
test_masks = [load_mask(mask_path) for mask_path in test_paths]
test_images = np.array(test_images)
test_masks = np.array(test_masks)

In [23]:

def unet_test_model(classes: int = 1):
    tf.keras.backend.clear_session()
    nb_filter = [32, 64, 128, 256, 512]
    # Build U-Net++ model
    inputs = Input((WIDTH, HEIGHT, 3))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.5)(c1)
    c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = Dropout(0.5)(c1)
    p1 = MaxPooling2D((2, 2), strides=(2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.5)(c2)
    c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = Dropout(0.5)(c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2))(c2)

    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(c2)
    conv1_2 = concatenate([up1_2, c1], name='merge12', axis=3)
    c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_2)
    c3 = Dropout(0.5)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = Dropout(0.5)(c3)

    conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    conv3_1 = Dropout(0.5)(conv3_1)
    conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_1)
    conv3_1 = Dropout(0.5)(conv3_1)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, c2], name='merge22', axis=3)  #x10
    conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_2)
    conv2_2 = Dropout(0.5)(conv2_2)
    conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_2)
    conv2_2 = Dropout(0.5)(conv2_2)

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, c1, c3], name='merge13', axis=3)
    conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_3)
    conv1_3 = Dropout(0.5)(conv1_3)
    conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_3)
    conv1_3 = Dropout(0.5)(conv1_3)

    conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4_1 = Dropout(0.5)(conv4_1)
    conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_1)
    conv4_1 = Dropout(0.5)(conv4_1)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3)  #x20
    conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_2)
    conv3_2 = Dropout(0.5)(conv3_2)
    conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_2)
    conv3_2 = Dropout(0.5)(conv3_2)

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, c2, conv2_2], name='merge23', axis=3)
    conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_3)
    conv2_3 = Dropout(0.5)(conv2_3)
    conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_3)
    conv2_3 = Dropout(0.5)(conv2_3)

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, c1, c3, conv1_3], name='merge14', axis=3)
    conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_4)
    conv1_4 = Dropout(0.5)(conv1_4)
    conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_4)
    conv1_4 = Dropout(0.5)(conv1_4)

    conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5_1 = Dropout(0.5)(conv5_1)
    conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv5_1)
    conv5_1 = Dropout(0.5)(conv5_1)

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3)  #x30
    conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_2)
    conv4_2 = Dropout(0.5)(conv4_2)
    conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_2)
    conv4_2 = Dropout(0.5)(conv4_2)

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
    conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_3)
    conv3_3 = Dropout(0.5)(conv3_3)
    conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_3)
    conv3_3 = Dropout(0.5)(conv3_3)

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], name='merge24', axis=3)
    conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_4)
    conv2_4 = Dropout(0.5)(conv2_4)
    conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_4)
    conv2_4 = Dropout(0.5)(conv2_4)

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, c1, c3, conv1_3, conv1_4], name='merge15', axis=3)
    conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_5)
    conv1_5 = Dropout(0.5)(conv1_5)
    conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_5)
    conv1_5 = Dropout(0.5)(conv1_5)

    nestnet_output_4 = Conv2D(classes, (1, 1), activation='sigmoid', kernel_initializer='he_normal', name='output_4',
                              padding='same')(conv1_5)
    # output = Conv2D(
    #     filters=classes,
    #     kernel_size=(3, 3),
    #     padding='same',
    #     activation='softmax',
    #     use_bias=True,
    #     kernel_initializer='glorot_uniform',
    #     name='final_conv',
    # )(nestnet_output_4)

    # create keras model instance
    return Model(inputs=[inputs], outputs=
    [nestnet_output_4])


In [24]:
LR = 0.0001


def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return 0.5 * tf.keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

In [25]:
from keras.losses import binary_crossentropy
import keras.backend as K


def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def weighted_dice_coeff(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    return score


def weighted_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    print(kernel_size)
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = 1 - weighted_dice_coeff(y_true, y_pred, weight)
    return loss


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
           (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


import tensorflow.keras.backend as K

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    # Остальной код функции...
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    print("Size ", kernel_size)
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + (1 - weighted_dice_coeff(y_true, y_pred, weight))
    return loss

In [26]:
print(len(result[0][0][0]))

NameError: ignored

In [27]:
import tensorflow as tf
from tensorflow.keras.metrics import IoU

model = unet_test_model(CLASSES)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=bce_dice_loss,
              metrics=[IoU(num_classes=CLASSES + 1, target_class_ids=
              [i for i in range(CLASSES + 1)])])

In [28]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=1, batch_size=16)


23/23 [==============================] - 83s 2s/step - loss: 4.2589 - io_u: 0.4486 - val_loss: 1.6029 - val_io_u: 0.4586


In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=5, batch_size=16)


Epoch 1/5
23/23 [==============================] - 27s 1s/step - loss: 2.1080 - io_u: 0.4554 - val_loss: 1.2533 - val_io_u: 0.4586
Epoch 2/5
23/23 [==============================] - 27s 1s/step - loss: 1.7742 - io_u: 0.4553 - val_loss: 1.2080 - val_io_u: 0.4586
Epoch 3/5
23/23 [==============================] - 27s 1s/step - loss: 1.5762 - io_u: 0.4553 - val_loss: 1.1971 - val_io_u: 0.4586
Epoch 4/5
23/23 [==============================] - 27s 1s/step - loss: 1.4538 - io_u: 0.4553 - val_loss: 1.1804 - val_io_u: 0.4586
Epoch 5/5
 3/23 [==>...........................] - ETA: 22s - loss: 1.4014 - io_u: 0.4567

In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=16)

In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=16)

In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=16)

In [ ]:
tf.saved_model.save(model, 'model')

In [ ]:
result = model.predict(test_images)

In [ ]:
print(result[0])

In [ ]:
from PIL import Image
import numpy as np

# Предположим, что результат предсказания модели содержит вероятности пикселей для каждого класса
# Размерность результатов предсказания: (batch_size, HEIGHT, WIDTH, num_classes)

# Предположим, что результат предсказания модели содержит вероятности пикселей для каждого класса
# Размерность результатов предсказания: (batch_size, HEIGHT, WIDTH, num_classes)
num_classes = 7  # Количество классов (6 классов + 1 класс "ничего")
threshold = 0.5  # Порог для принятия решения о наличии объекта на пикселе

# Преобразование результатов предсказания в маски с максимальными значениями классов
part = 100
predicted_masks = result[:part]
# true_masks = test_masks
# Создание палитры цветов для каждого класса
palette = [
    (0, 0, 0),  # Класс "ничего" (черный цвет)
    (255, 0, 0),  # Класс 1 (красный цвет)
    (0, 255, 0),  # Класс 2 (зеленый цвет)
    (0, 0, 255),  # Класс 3 (синий цвет)
    (255, 255, 0),  # Класс 4 (желтый цвет)
    (255, 0, 255),  # Класс 5 (пурпурный цвет)
    (0, 255, 255),  # Класс 6 (бирюзовый цвет)
    (255, 255, 255)
]
#
# Создание изображений для каждого предсказанного класса
for i in range(min(predicted_masks.shape[0], part)):
    image = Image.new("RGB",
                      (
                          WIDTH, HEIGHT
                      )
                      )
    # true_mask = Image.new("RGB",
    #                   (
    #                       WIDTH, HEIGHT
    #                   )
    #                   )
    for y in range(
            HEIGHT
    ):
        for x in range(
                WIDTH
        ):
            if predicted_masks[i][y][x] >= threshold:
                class_index = 7
            else:
                class_index = 0
            # val = predicted_masks[i][x][y]
            image.putpixel((x, y), palette[class_index])  #(int(val * 255), int(val * 255), int(val * 255)))
            # if true_masks[i][y][x] >= threshold:
            #     true_mask.putpixel((x, y), palette[7])
            # else:
            #     true_mask.putpixel((x, y), palette[0])
    image.save(f"result/predicted_image_{i}.png")
    # true_mask.save(f"result_small_5/predicted_image_{i}_true.png")



In [ ]:
print(test_masks[0])